In [21]:
import geopandas as gpd
import os
os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')

pop_density=gpd.read_file('Population_Density_(Census_Tracts)')
flowlines=gpd.read_file('matched_flowlines.geojson')

In [22]:
# Check the format of the population data - this code uses the census tract data available at:
# https://data-cdphe.opendata.arcgis.com/datasets/CDPHE::population-density-census-tracts/explore?location=38.499827%2C-102.988618%2C6.77

print('Summary of Census Tract Data:')
print(pop_density.info())  # General information about the dataset
print('\nFirst few rows of the data:')
print(pop_density.head())  # Preview the first few rows

Summary of Census Tract Data:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    1249 non-null   int32   
 1   FIPS        1249 non-null   object  
 2   County      1249 non-null   object  
 3   Tract_Name  1249 non-null   object  
 4   Area_Land_  1249 non-null   float64 
 5   Population  1249 non-null   int32   
 6   Populati_1  1249 non-null   float64 
 7   geometry    1249 non-null   geometry
dtypes: float64(2), geometry(1), int32(2), object(3)
memory usage: 68.4+ KB
None

First few rows of the data:
   OBJECTID         FIPS    County  \
0         1  08043979000   FREMONT   
1         2  08045951600  GARFIELD   
2         3  08069002803   LARIMER   
3         4  08125963200      YUMA   
4         5  08069002401   LARIMER   

                                     Tract_Name  Area_Land_  Population  \
0   Census Tract 9790,

In [23]:
print(pop_density['Populati_1'].describe())


count     1249.000000
mean      3664.680384
std       3641.594843
min          0.000000
25%        466.600000
50%       3139.000000
75%       5487.300000
max      33066.700000
Name: Populati_1, dtype: float64


In [24]:
# Ensure matching coordinates

pop_density = pop_density.to_crs(flowlines.crs)

print(f'Flowline CRS: {flowlines.crs}')
print(f'Population CRS: {pop_density.crs}')



Flowline CRS: EPSG:26913
Population CRS: EPSG:26913


In [25]:
# Create a buffer around each line in which to calculate population density
# If a buffer lies within multiple tracts, we will take the average density between the tracts.
# A better approach would be an average weighted by the proportion of the buffer in each tract, but I am still working on this.
# It may be possible that our lines are cut into small enough chunks that this isn't an issue.

buffer_distance = 10  # buffer distance in meters
flowlines['buffer'] = flowlines.geometry.buffer(buffer_distance)

buffered_flowlines = gpd.GeoDataFrame(flowlines.drop(columns='geometry'), geometry=flowlines["buffer"])  # create a new dataframe that uses the buffered geometry
buffered_flowlines = buffered_flowlines.reset_index()  # reset index columns

# Perform a left spatial join with the updated predicate parameter
joined = gpd.sjoin(buffered_flowlines, pop_density, how='left', predicate='intersects')

In [26]:
# test how many of the buffers intersect at least one tract

intersection_test = buffered_flowlines.geometry.intersects(pop_density.geometry.unary_union)
print(intersection_test.sum())  

/var/folders/p3/gs65vvgs1wd90mnxb3q3hvy80000gp/T/ipykernel_9195/1035208378.py:3: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersection_test = buffered_flowlines.geometry.intersects(pop_density.geometry.unary_union)


14922


In [27]:
# calculate average density for each buffered line and put it in a new column

aggregated_density = (joined.groupby("index")["Populati_1"].mean().reset_index(name="average_pop_density")) 

flowlines=flowlines.merge(aggregated_density, left_index=True, right_on='index', how='left')


In [28]:
flowlines

,ACTIONDESCRIPTION,BEDDINGMATERIAL,COMPANY_NAME,CONSTRUCTDATE,Diam_in,ENDLAT,ENDLONG,ENTIRELINEREMOVED,FLOWLINEACTION,FLOWLINEID,...,SHAPE_Length,STARTLAT,STARTLOCATIONID,STARTLONG,Status,TYPEOFFLUIDTRANS,geometry,buffer,index,average_pop_density
0,The flowline serving the Emerson 3-29J (05-123...,Native Materials,CRESTONE PEAK RESOURCES OPERATING LLC ...,1983-11-09,2.0,40.109444,-104.909686,None,None,470446.0,...,414.751530,40.105743,NaN,-104.909860,Active,Multiphase,"MULTILINESTRING ((507682.447 4439497.657, 5076...","POLYGON ((507673.543 4439514.384, 507673.728 4...",0,549.8
1,The flowline (12311399_FL) servicing the Emers...,Native Materials,CRESTONE PEAK RESOURCES OPERATING LLC ...,1983-12-07,2.0,40.109441,-104.909670,True,Out of Service,470445.0,...,312.594254,40.112203,319521.0,-104.909862,Active,Multiphase,"MULTILINESTRING ((507681.553 4440214.644, 5076...","POLYGON ((507699.365 4440209.498, 507699.952 4...",1,549.8
2,None,None,PETERSON ENERGY OPERATING INC ...,2006-06-05,2.0,40.063138,-105.034086,None,None,466061.0,...,84.099157,40.063823,305594.0,-105.030068,Active,Multiphase,"MULTILINESTRING ((497028.859 4434818.938, 4970...","POLYGON ((497096.076 4434767.416, 497096.745 4...",2,173.4
3,None,None,PETERSON ENERGY OPERATING INC ...,2007-04-08,2.0,40.063142,-105.034085,None,None,466062.0,...,84.099157,40.060063,305595.0,-105.034795,Active,Multiphase,"MULTILINESTRING ((497028.859 4434818.938, 4970...","POLYGON ((497096.076 4434767.416, 497096.745 4...",3,173.4
4,None,None,PETERSON ENERGY OPERATING INC ...,2005-05-10,2.0,40.063132,-105.034086,None,None,466063.0,...,84.099157,40.059581,332338.0,-105.029331,Active,Multiphase,"MULTILINESTRING ((497028.859 4434818.938, 4970...","POLYGON ((497096.076 4434767.416, 497096.745 4...",4,173.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14917,None,None,NOBLE ENERGY INC ...,2001-10-02,3.0,40.397700,-104.647660,None,Registration,455232.0,...,430.739655,40.398090,333012.0,-104.642610,Abandoned,None,"MULTILINESTRING ((530328.842 4472004.142, 5299...","POLYGON ((529901.505 4471949.187, 529900.525 4...",14917,3196.7
14918,None,None,NOBLE ENERGY INC ...,2008-02-22,2.0,40.481530,-104.492930,None,Registration,455190.0,...,160.751553,40.481330,310572.0,-104.491050,Active,None,"MULTILINESTRING ((543138.306 4481306.544, 5429...","POLYGON ((542977.624 4481317.604, 542976.659 4...",14918,2.3
14919,None,None,NOBLE ENERGY INC ...,2007-08-05,3.0,40.495330,-104.772282,None,Registration,455155.0,...,630.836779,40.489670,306660.0,-104.771610,Abandoned,None,"MULTILINESTRING ((519355.342 4482133.153, 5192...","POLYGON ((519286.813 4482760.336, 519286.77 44...",14919,112.4
14920,None,None,NOBLE ENERGY INC ...,1998-09-27,2.0,40.495330,-104.772282,None,Registration,455156.0,...,243.837753,40.493320,306575.0,-104.770310,Active,None,"MULTILINESTRING ((519465.329 4482537.89, 51931...","POLYGON ((519309.599 4482725.786, 519309.043 4...",14920,112.4


In [29]:
# Save the updated file
flowlines.drop(columns=['buffer', 'index']).to_file(
    'flowlines_with_pop_density.geojson',
    driver='GeoJSON'
)


In [30]:
flowlines_with_pop_density=gpd.read_file('flowlines_with_pop_density.geojson')

In [31]:
flowlines_with_pop_density.head()

,ACTIONDESCRIPTION,BEDDINGMATERIAL,COMPANY_NAME,CONSTRUCTDATE,Diam_in,ENDLAT,ENDLONG,ENTIRELINEREMOVED,FLOWLINEACTION,FLOWLINEID,...,PIPEMATERIAL,RECEIVE_DATE,SHAPE_Length,STARTLAT,STARTLOCATIONID,STARTLONG,Status,TYPEOFFLUIDTRANS,average_pop_density,geometry
0,The flowline serving the Emerson 3-29J (05-123...,Native Materials,CRESTONE PEAK RESOURCES OPERATING LLC ...,1983-11-09,2.0,40.109444,-104.909686,None,None,470446.0,...,Carbon Steel,2023-10-24 08:33:49.223,414.751530,40.105743,NaN,-104.909860,Active,Multiphase,549.8,"MULTILINESTRING ((507682.447 4439497.657, 5076..."
1,The flowline (12311399_FL) servicing the Emers...,Native Materials,CRESTONE PEAK RESOURCES OPERATING LLC ...,1983-12-07,2.0,40.109441,-104.909670,True,Out of Service,470445.0,...,Carbon Steel,2023-10-24 08:33:49.223,312.594254,40.112203,319521.0,-104.909862,Active,Multiphase,549.8,"MULTILINESTRING ((507681.553 4440214.644, 5076..."
2,None,None,PETERSON ENERGY OPERATING INC ...,2006-06-05,2.0,40.063138,-105.034086,None,None,466061.0,...,Carbon Steel,2023-10-24 16:00:01.117,84.099157,40.063823,305594.0,-105.030068,Active,Multiphase,173.4,"MULTILINESTRING ((497028.859 4434818.938, 4970..."
3,None,None,PETERSON ENERGY OPERATING INC ...,2007-04-08,2.0,40.063142,-105.034085,None,None,466062.0,...,Carbon Steel,2023-10-24 16:00:01.117,84.099157,40.060063,305595.0,-105.034795,Active,Multiphase,173.4,"MULTILINESTRING ((497028.859 4434818.938, 4970..."
4,None,None,PETERSON ENERGY OPERATING INC ...,2005-05-10,2.0,40.063132,-105.034086,None,None,466063.0,...,Carbon Steel,2023-10-24 16:00:01.117,84.099157,40.059581,332338.0,-105.029331,Active,Multiphase,173.4,"MULTILINESTRING ((497028.859 4434818.938, 4970..."
